In [36]:
import pandas as pd

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

In [37]:
def format_time(x):
    if not any(i in x for i in ['DNF', 'DNS']):
        if ':' in x:
            return round(float(str(x).split(':')[1]) + (60 * float(str(x).split(':')[0])), 3) if x != 0 else 0
        else:
            return(round(float(x), 3))
    else:
        return x

In [38]:
races = pd.read_csv(path+'data/races.csv')
qualifying = pd.read_csv(path+'data/qualifying.csv')
constructors = pd.read_csv(path+'data/constructor_standings.csv')
drivers = pd.read_csv(path+'data/driver_standings.csv')
results = pd.read_csv(path+'data/results.csv')
starting_grid = pd.read_csv(path+'data/starting-grid.csv')

### Redundant until new weather collection method is implemented
# weather = pd.read_csv(path+'data/weather.csv')

In [23]:
laps1 = pd.read_csv(path+'data/laps-2014-2017.csv')
laps2 = pd.read_csv(path+'data/laps-2017-22.csv')

In [5]:
laps = pd.concat([laps1, laps2]).drop_duplicates()

laps.shape

(168937, 6)

In [6]:
laps[laps.duplicated()]

,season,round,lap,driver,position,time


In [7]:
laps['time'] = laps.time.apply(lambda x: format_time(str(x)))

In [8]:
# laps.to_csv(path+'data/laps.csv', index=False)

In [39]:
def lookup (df, team, points):
    df['lookup1'] = df.season.astype(str) + df[team] + df['round'].astype(str)
    df['lookup2'] = df.season.astype(str) + df[team] + (df['round']-1).astype(str)
    new_df = df.merge(df[['lookup1', points]], how = 'left', left_on='lookup2',right_on='lookup1')
    new_df.drop(['lookup1_x', 'lookup2', 'lookup1_y'], axis = 1, inplace = True)
    new_df.rename(columns = {points+'_x': points+'_after_race', points+'_y': points}, inplace = True)
    new_df[points].fillna(0, inplace = True)
    return new_df

In [40]:
for col in ['driver_points', 'driver_wins', 'driver_standings_pos']:
    drivers = lookup(drivers, 'driver', col)

# drivers.drop(['driver_points_after_race', 'driver_wins_after_race', 'driver_standings_pos_after_race'], axis=1, inplace=True)

In [41]:
for col in ['constructor_points', 'constructor_wins', 'constructor_standings_pos']:
    constructors = lookup(constructors, 'constructor', col)

# constructors.drop(['constructor_points_after_race', 'constructor_wins_after_race', 'constructor_standings_pos_after_race'], axis=1, inplace=True)

In [11]:
mean_time = laps.groupby(['season', 'round', 'driver']).agg({'time': 'mean'}).reset_index()
mean_time['time'] = mean_time.time.apply(lambda x: round(x, 3))

NameError: name 'laps' is not defined

In [56]:
df1 = pd.merge(races, results, how='inner', on=['season', 'round', 'circuit_id']).drop(['url','points', 'status', 'time', 'grid'], axis=1)
df2 = pd.merge(df1, drivers, how='left', on=['season', 'round', 'driver']) 
df3 = pd.merge(df2, constructors, how='left', on=['season', 'round', 'constructor'])
df4 = pd.merge(df3, starting_grid, how='inner', on=['season', 'round', 'driver', 'constructor'])
df4.rename(columns = {'grid': 'starting_grid'}, inplace=True)

merged_df = pd.merge(df4, qualifying, how='inner', on=['season', 'round', 'circuit_id', 'driver'])
merged_df.rename(columns = {'fastest_time': 'qual_time', 'grid': 'qualifying_pos'}, inplace=True)

merged_df.shape

(0, 30)

In [63]:
df3

,season,round,circuit_id,country,lat,long,date,driver,date_of_birth,nationality,...,driver_standings_pos_after_race,driver_points,driver_wins,driver_standings_pos,constructor_points_after_race,constructor_wins_after_race,constructor_standings_pos_after_race,constructor_points,constructor_wins,constructor_standings_pos
0,2014,1,albert_park,Australia,-37.8497,144.9680,2014-03-16 06:00:00,rosberg,1985-06-27,German,...,1,0.0,0.0,0.0,25.0,1,2,0.0,0.0,0.0
1,2014,1,albert_park,Australia,-37.8497,144.9680,2014-03-16 06:00:00,kevin_magnussen,1992-10-05,Danish,...,2,0.0,0.0,0.0,33.0,0,1,0.0,0.0,0.0
2,2014,1,albert_park,Australia,-37.8497,144.9680,2014-03-16 06:00:00,button,1980-01-19,British,...,3,0.0,0.0,0.0,33.0,0,1,0.0,0.0,0.0
3,2014,1,albert_park,Australia,-37.8497,144.9680,2014-03-16 06:00:00,alonso,1981-07-29,Spanish,...,4,0.0,0.0,0.0,18.0,0,3,0.0,0.0,0.0
4,2014,1,albert_park,Australia,-37.8497,144.9680,2014-03-16 06:00:00,bottas,1989-08-28,Finnish,...,5,0.0,0.0,0.0,10.0,0,4,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3702,2022,22,yas_marina,UAE,24.4672,54.6031,2022-11-20 13:00:00,mick_schumacher,1999-03-22,German,...,16,12.0,0.0,16.0,37.0,0,8,37.0,0.0,8.0
3703,2022,22,yas_marina,UAE,24.4672,54.6031,2022-11-20 13:00:00,kevin_magnussen,1992-10-05,Danish,...,13,25.0,0.0,13.0,37.0,0,8,37.0,0.0,8.0
3704,2022,22,yas_marina,UAE,24.4672,54.6031,2022-11-20 13:00:00,hamilton,1985-01-07,British,...,6,240.0,0.0,5.0,515.0,1,3,505.0,1.0,3.0
3705,2022,22,yas_marina,UAE,24.4672,54.6031,2022-11-20 13:00:00,latifi,1995-06-29,Canadian,...,20,2.0,0.0,20.0,8.0,0,10,8.0,0.0,10.0


In [60]:
df3.shape

(3707, 25)

In [43]:
merged_df['fastest_lap'] = merged_df['fastest_lap'].apply(lambda x: format_time(str(x)))

In [44]:
merged_df[['fastest_lap', 'qual_time']].head()

,fastest_lap,qual_time
0,92.478,104.595
1,93.066,105.745
2,92.917,104.437
3,93.186,105.819
4,92.616,108.147


In [45]:
merged_df[['season','round', 'driver', 'starting_grid', 'qualifying_pos','podium', 'fastest_lap']].query('season == 2015 & round == 1').sort_values('starting_grid')

,season,round,driver,starting_grid,qualifying_pos,podium,fastest_lap
405,2015,1,hamilton,NaN,1,1,90.945
406,2015,1,rosberg,NaN,2,2,91.092
407,2015,1,vettel,NaN,4,3,91.457
408,2015,1,massa,NaN,3,4,91.719
409,2015,1,nasr,NaN,11,5,92.612
410,2015,1,ricciardo,NaN,7,6,92.797
411,2015,1,hulkenberg,NaN,14,7,91.970
412,2015,1,ericsson,NaN,16,8,91.560
413,2015,1,sainz,NaN,8,9,92.872
414,2015,1,perez,NaN,15,10,91.959


In [31]:
merged_df.columns

Index(['season', 'round', 'circuit_id', 'country', 'lat', 'long', 'date',
       'driver', 'date_of_birth', 'nationality', 'constructor', 'podium',
       'fastest_lap', 'driver_points_after_race', 'driver_wins_after_race',
       'driver_standings_pos_after_race', 'driver_points', 'driver_wins',
       'driver_standings_pos', 'constructor_points_after_race',
       'constructor_wins_after_race', 'constructor_standings_pos_after_race',
       'constructor_points', 'constructor_wins', 'constructor_standings_pos',
       'starting_grid', 'qualifying_pos', 'qual_time', 'stage', 'q_delta'],
      dtype='object')

In [32]:
null_cols = merged_df.columns[merged_df.isna().any()].tolist()
# merged_df.query()
null_rows = merged_df[merged_df.isna().any(axis=1)]
null_rows = null_rows[['season', 'round', 'driver', 'podium', 'qual_time','fastest_lap', 'stage']]

In [33]:
null_rows

,season,round,driver,podium,qual_time,fastest_lap,stage
0,2014,1,rosberg,1,104.595,92.478,q3
1,2014,1,kevin_magnussen,2,105.745,93.066,q3
2,2014,1,button,3,104.437,92.917,q2
3,2014,1,alonso,4,105.819,93.186,q3
4,2014,1,bottas,5,108.147,92.616,q3
...,...,...,...,...,...,...,...
3666,2022,21,latifi,16,75.095,75.327,q1
3667,2022,21,tsunoda,17,76.264,75.508,q1
3668,2022,21,norris,18,72.263,75.855,q3
3669,2022,21,kevin_magnussen,19,71.674,NaN,q3


In [35]:
merged_df

,season,round,circuit_id,country,lat,long,date,driver,date_of_birth,nationality,...,constructor_wins_after_race,constructor_standings_pos_after_race,constructor_points,constructor_wins,constructor_standings_pos,starting_grid,qualifying_pos,qual_time,stage,q_delta


In [34]:
# fill/drop nulls
### Data is being lossed here

for col in ['driver_points', 'driver_wins', 'driver_standings_pos', 'constructor_points', 
            'constructor_wins', 'constructor_standings_pos']:
    merged_df[col].fillna(0, inplace=True)
    merged_df[col] = merged_df[col].map(lambda x: int(x))
    
merged_df.dropna(inplace=True)

merged_df.shape

(0, 30)

In [33]:
# from dateutil.relativedelta import *

# merged_df['date_of_birth'] = pd.to_datetime(merged_df.date_of_birth)
# merged_df['driver_age'] = merged_df.apply(lambda x: relativedelta(pd.to_datetime(x['date']), x['date_of_birth']).years, axis=1)
# merged_df.drop(['date_of_birth'], axis=1, inplace = True)

ValueError: Cannot set a DataFrame with multiple columns to the single column driver_age

In [20]:
merged_df.to_csv(path+'data/merged.csv', index=False)